Importing Requried Modules

In [1]:
import numpy as np 
import pandas as pd 
import matplotlib.pyplot as plt
import seaborn as sns 
import nltk
from nltk.corpus import stopwords
import re 
import warnings 
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics import accuracy_score , f1_score  

In [2]:
warnings.filterwarnings ("ignore")

In [3]:
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\nagar\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [4]:
print(stopwords.words('english'))

['a', 'about', 'above', 'after', 'again', 'against', 'ain', 'all', 'am', 'an', 'and', 'any', 'are', 'aren', "aren't", 'as', 'at', 'be', 'because', 'been', 'before', 'being', 'below', 'between', 'both', 'but', 'by', 'can', 'couldn', "couldn't", 'd', 'did', 'didn', "didn't", 'do', 'does', 'doesn', "doesn't", 'doing', 'don', "don't", 'down', 'during', 'each', 'few', 'for', 'from', 'further', 'had', 'hadn', "hadn't", 'has', 'hasn', "hasn't", 'have', 'haven', "haven't", 'having', 'he', "he'd", "he'll", 'her', 'here', 'hers', 'herself', "he's", 'him', 'himself', 'his', 'how', 'i', "i'd", 'if', "i'll", "i'm", 'in', 'into', 'is', 'isn', "isn't", 'it', "it'd", "it'll", "it's", 'its', 'itself', "i've", 'just', 'll', 'm', 'ma', 'me', 'mightn', "mightn't", 'more', 'most', 'mustn', "mustn't", 'my', 'myself', 'needn', "needn't", 'no', 'nor', 'not', 'now', 'o', 'of', 'off', 'on', 'once', 'only', 'or', 'other', 'our', 'ours', 'ourselves', 'out', 'over', 'own', 're', 's', 'same', 'shan', "shan't", 'she

Loading Dataset

In [5]:
dataset=pd.read_csv("training.1600000.processed.noemoticon.csv", encoding ="ISO-8859-1")
dataset.head()

,0,1467810369,Mon Apr 06 22:19:45 PDT 2009,NO_QUERY,_TheSpecialOne_,"@switchfoot http://twitpic.com/2y1zl - Awww, that's a bummer. You shoulda got David Carr of Third Day to do it. ;D"
0,0,1467810672,Mon Apr 06 22:19:49 PDT 2009,NO_QUERY,scotthamilton,is upset that he can't update his Facebook by ...
1,0,1467810917,Mon Apr 06 22:19:53 PDT 2009,NO_QUERY,mattycus,@Kenichan I dived many times for the ball. Man...
2,0,1467811184,Mon Apr 06 22:19:57 PDT 2009,NO_QUERY,ElleCTF,my whole body feels itchy and like its on fire
3,0,1467811193,Mon Apr 06 22:19:57 PDT 2009,NO_QUERY,Karoli,"@nationwideclass no, it's not behaving at all...."
4,0,1467811372,Mon Apr 06 22:20:00 PDT 2009,NO_QUERY,joy_wolf,@Kwesidei not the whole crew


In [6]:
col_names = ['target' , 'id' , 'date' , 'flag' , 'user' , 'text']
dataset.columns = col_names

In [7]:
dataset.head()

,target,id,date,flag,user,text
0,0,1467810672,Mon Apr 06 22:19:49 PDT 2009,NO_QUERY,scotthamilton,is upset that he can't update his Facebook by ...
1,0,1467810917,Mon Apr 06 22:19:53 PDT 2009,NO_QUERY,mattycus,@Kenichan I dived many times for the ball. Man...
2,0,1467811184,Mon Apr 06 22:19:57 PDT 2009,NO_QUERY,ElleCTF,my whole body feels itchy and like its on fire
3,0,1467811193,Mon Apr 06 22:19:57 PDT 2009,NO_QUERY,Karoli,"@nationwideclass no, it's not behaving at all...."
4,0,1467811372,Mon Apr 06 22:20:00 PDT 2009,NO_QUERY,joy_wolf,@Kwesidei not the whole crew


In [8]:
dataset.shape

(1599999, 6)

In [9]:
dataset.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1599999 entries, 0 to 1599998
Data columns (total 6 columns):
 #   Column  Non-Null Count    Dtype 
---  ------  --------------    ----- 
 0   target  1599999 non-null  int64 
 1   id      1599999 non-null  int64 
 2   date    1599999 non-null  object
 3   flag    1599999 non-null  object
 4   user    1599999 non-null  object
 5   text    1599999 non-null  object
dtypes: int64(2), object(4)
memory usage: 73.2+ MB


In [10]:
dataset.isnull().sum() # checking for null values 

target    0
id        0
date      0
flag      0
user      0
text      0
dtype: int64

In [12]:
dataset['target'].value_counts()

target
4    800000
0    799999
Name: count, dtype: int64

Data Preprocessing

In [14]:
# replacing 4 with 1 
dataset['target']=dataset['target'].map({4:1,0:0})

In [17]:
#1 for positive tweets and 0 for negetive 
dataset['target'].value_counts()

target
1    800000
0    799999
Name: count, dtype: int64

In [18]:
print(dataset['text']) 

0          is upset that he can't update his Facebook by ...
1          @Kenichan I dived many times for the ball. Man...
2            my whole body feels itchy and like its on fire 
3          @nationwideclass no, it's not behaving at all....
4                              @Kwesidei not the whole crew 
                                 ...                        
1599994    Just woke up. Having no school is the best fee...
1599995    TheWDB.com - Very cool to hear old Walt interv...
1599996    Are you ready for your MoJo Makeover? Ask me f...
1599997    Happy 38th Birthday to my boo of alll time!!! ...
1599998    happy #charitytuesday @theNSPCC @SparksCharity...
Name: text, Length: 1599999, dtype: object


In [21]:
from nltk.stem.porter import PorterStemmer
stemmer=PorterStemmer()
def stemming(tweet):
    stemmed_tweet=re.sub('[^a-zA-Z]'," ", tweet)
    stemmed_tweet=stemmed_tweet.lower()
    stemmed_tweet=stemmed_tweet.split()
    stemmed_tweet=[stemmer.stem(word) for word in stemmed_tweet if not word in stopwords.words('english')]
    stemmed_tweet=" ".join(stemmed_tweet)
    return stemmed_tweet

In [22]:
dataset['text'] = dataset['text'].apply(stemming)

In [24]:
print(dataset['text']) # after stemming

0          upset updat facebook text might cri result sch...
1          kenichan dive mani time ball manag save rest g...
2                            whole bodi feel itchi like fire
3                              nationwideclass behav mad see
4                                        kwesidei whole crew
                                 ...                        
1599994                           woke school best feel ever
1599995    thewdb com cool hear old walt interview http b...
1599996                         readi mojo makeov ask detail
1599997    happi th birthday boo alll time tupac amaru sh...
1599998    happi charitytuesday thenspcc sparkschar speak...
Name: text, Length: 1599999, dtype: object


Splitting Inputs 

In [27]:
x = dataset['text']
y = dataset['target']

In [29]:
x_train , x_test , y_train , y_test = train_test_split(x , y , test_size = 0.2 , random_state = 0)

In [31]:
# convert textual data to numerical data
vectorizer = TfidfVectorizer()
x_train = vectorizer.fit_transform(x_train)
x_test = vectorizer.transform(x_test)

In [32]:
print(x_train)

<Compressed Sparse Row sparse matrix of dtype 'float64'
	with 9451734 stored elements and shape (1279999, 460909)>
  Coords	Values
  (0, 443991)	0.22625223143666687
  (0, 150562)	0.187752051036393
  (0, 149660)	0.12602103676347354
  (0, 393595)	0.18633353695642413
  (0, 246262)	0.516206150117446
  (0, 422796)	0.4213995220282958
  (0, 160355)	0.18966194768681632
  (0, 384310)	0.38648598535906226
  (0, 145591)	0.48328892862950174
  (1, 418051)	0.7948992424350689
  (1, 172128)	0.6067414559564506
  (2, 42279)	0.5211994648067829
  (2, 150650)	0.20986098127991223
  (2, 290673)	0.24841016587340456
  (2, 275790)	0.3769717187165907
  (2, 406965)	0.6931768888241752
  (3, 175234)	0.4280552121498152
  (3, 317428)	0.5137960384023271
  (3, 399931)	0.21912347276618377
  (3, 292469)	0.3352332134067401
  (3, 135304)	0.18399221471225605
  (3, 89478)	0.5137960384023271
  (3, 175231)	0.30748407834013664
  (4, 153091)	0.1705989235646957
  (4, 93767)	0.13053894204186417
  :	:
  (1279995, 248952)	0.588995863

Training The Model 

In [36]:
from sklearn.linear_model import LogisticRegression 
model= LogisticRegression()
model.fit(x_train, y_train)

LogisticRegression()

In [73]:
pred=model.predict(x_test)
acc=accuracy_score(y_test,pred)

In [76]:
print("Your Accuracy Score is :" ,acc*100)

Your Accuracy Score is : 77.56875


In [37]:
# Function to predict the sentiment
def predict_sentiment(text):
    text = re.sub('[^a-zA-Z]',' ',text) # removing not a-z and A-Z
    text = text.lower()
    text = text.split() 
    text = [stemmer.stem(word) for word in text if not word in stopwords.words('english')]
    text = ' '.join(text)
    text = [text]
    text = vectorizer.transform(text)   
    sentiment = model.predict(text)
    if sentiment == 0:
        return "Negative"
    else:
        return "Positive"

In [1]:
print(predict_sentiment(" have a great day"))

NameError: name 'predict_sentiment' is not defined

In [68]:
import pickle
pickle.dump(model , open('model.pkl' , 'wb'))

In [69]:
pickle.dump(vectorizer , open('vectorizer.pkl' , 'wb'))